# AWS Glue Studio Notebook
##### You are now running a AWS Glue Studio notebook; To start using your notebook you need to start an AWS Glue Interactive Session.


#### Optional: Run this cell to see available notebook commands ("magics").


In [ ]:
%help

####  Run this cell to set up and start your interactive session.


In [ ]:
%idle_timeout 2880
%glue_version 5.0
%worker_type G.1X
%number_of_workers 5

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

#### Example: Create a DynamicFrame from a table in the AWS Glue Data Catalog and display its schema


In [ ]:
dyf = glueContext.create_dynamic_frame.from_catalog(database='database_name', table_name='table_name')
dyf.printSchema()

#### Example: Convert the DynamicFrame to a Spark DataFrame and display a sample of the data


In [ ]:
df = dyf.toDF()
df.show()

#### Example: Visualize data with matplotlib


In [ ]:
import matplotlib.pyplot as plt

# Set X-axis and Y-axis values
x = [5, 2, 8, 4, 9]
y = [10, 4, 8, 5, 2]
  
# Create a bar chart 
plt.bar(x, y)
  
# Show the plot
%matplot plt

#### Example: Write the data in the DynamicFrame to a location in Amazon S3 and a table for it in the AWS Glue Data Catalog


In [ ]:
s3output = glueContext.getSink(
  path="s3://bucket_name/folder_name",
  connection_type="s3",
  updateBehavior="UPDATE_IN_DATABASE",
  partitionKeys=[],
  compression="snappy",
  enableUpdateCatalog=True,
  transformation_ctx="s3output",
)
s3output.setCatalogInfo(
  catalogDatabase="demo", catalogTableName="populations"
)
s3output.setFormat("glueparquet")
s3output.writeFrame(DyF)

In [ ]:
import sys
import boto3
import pandas as pd

# Input arguments for the Glue job
args = sys.argv
source_bucket_name = "team11projectdw"  # Source S3 bucket for dimension tables
target_bucket_name = "team11projectdw"  # Target S3 bucket for SalesFact table
olap_prefix = "olap/"  # Folder for transformed data

# File paths for existing dimension tables
order_dimension_file = f"s3://{source_bucket_name}/{olap_prefix}order_dimension/order_dimension.csv"
review_dimension_file = f"s3://{source_bucket_name}/{olap_prefix}review_dimension/review_dimension.csv"
product_dimension_file = f"s3://{source_bucket_name}/{olap_prefix}product_dimension/product_dimension.csv"
seller_dimension_file = f"s3://{source_bucket_name}/{olap_prefix}seller_dimension/seller_dimension.csv"

# --- Step 1: Load the dimension data into Pandas DataFrames ---
print("Loading dimension tables from S3...")
df_order_dimension = pd.read_csv(order_dimension_file)
df_review_dimension = pd.read_csv(review_dimension_file)
df_product_dimension = pd.read_csv(product_dimension_file)
df_seller_dimension = pd.read_csv(seller_dimension_file)

# --- Step 2: Create SalesFact Table ---
print("Creating SalesFact table...")

# Merge dimensions
salesfact = df_order_dimension.merge(df_review_dimension, on='order_id', how='left') \
                               .merge(df_product_dimension, on='product_id', how='left') \
                               .merge(df_seller_dimension, on='seller_id', how='left')

# Calculate derived columns
salesfact['productvolume'] = (salesfact['product_length'] * 
                              salesfact['product_width'] * 
                              salesfact['product_height'])
salesfact['totalordervalue'] = salesfact['price'] + salesfact['freight_value']

# Select relevant columns for the SalesFact table
salesfact = salesfact[['Orderkey', 'Reviewkey', 'Productkey', 'Sellerkey', 
                      'price', 'freight_value', 'product_weight', 'product_length', 
                      'product_width', 'product_height', 'productvolume', 
                      'totalordervalue', 'score']]  # Adding reviewscore as 'score'

# Rename the column 'score' to 'reviewscore' for better clarity
salesfact.rename(columns={'score': 'reviewscore'}, inplace=True)

# --- Step 3: Check for duplicates and drop them ---
print("Checking for duplicates in SalesFact table...")
salesfact = salesfact.drop_duplicates().reset_index(drop=True)

# --- Step 4: Save Transformed SalesFact Data Locally ---
print("Saving SalesFact data locally...")
salesfact.to_csv("/tmp/salesfact.csv", index=False)

# --- Step 5: Upload SalesFact Data to S3 ---
print("Uploading SalesFact data to S3...")
s3 = boto3.client('s3')
s3.upload_file("/tmp/salesfact.csv", target_bucket_name, f"{olap_prefix}salesfact/salesfact.csv")

print("SalesFact table created and uploaded successfully!")
